In [5]:
import pandas as pd
from pyecharts.charts import Bar, Line
from pyecharts import options as opts
import os

def get_10min_data_for_file(filename):
    """
    辅助函数：从指定文件中提取距离起始时间10分钟左右的数据。
    """
    if not os.path.exists(filename):
        print(f"文件 {filename} 未找到。")
        return None

    try:
        df = pd.read_csv(filename)
        
        if df.empty:
            print(f"文件 {filename} 为空。")
            return None

        df['timestamp'] = pd.to_datetime(df['timestamp'])

        min_ts = df['timestamp'].min()
        
        # 计算目标时间点：起始时间 + 10分钟
        target_time = min_ts + pd.Timedelta(minutes=10)
        
        # 计算每个记录距离目标时间点的绝对时间差
        df['time_diff_from_target'] = abs(df['timestamp'] - target_time)
        
        # 找到最小的时间差
        min_time_diff = df['time_diff_from_target'].min()
        
        # 提取所有具有最小时间差的行（即最接近10分钟时间点的所有peer数据）
        # 这样可以确保即使有多个peer在同一个最接近10分钟的时间点记录，也能全部获取
        final_df = df[df['time_diff_from_target'] == min_time_diff].copy()

        if final_df.empty:
            print(f"无法从文件 {filename} 中提取第十分钟的数据。")
            return None
            
        # 获取实际被选中的时间戳，用于图表标题
        actual_selected_ts = final_df['timestamp'].iloc[0] 
        
        return final_df, actual_selected_ts

    except Exception as e:
        print(f"处理文件 {filename} 时发生错误: {e}")
        return None

def plot_redundancy_at_10min(fanouts, output_dir="redundancy_charts"):
    """
    处理指定扇出值的CSV文件，提取第十分钟的peer冗余数据，并生成Pyecharts图表。
    """
    os.makedirs(output_dir, exist_ok=True)
    print(f"图表将保存到目录: {os.path.abspath(output_dir)}\n")

    for fanout in fanouts:
        filename = f"redundancy_data_fanout_{fanout}.csv"
        print(f"正在处理文件: {filename}...")

        result = get_10min_data_for_file(filename)
        if result is None:
            print(f"跳过文件 {filename}。\n")
            continue
        
        final_df, actual_selected_ts = result

        # 按peer_id排序，使图表显示顺序一致
        final_df = final_df.sort_values(by='peer_id')

        peer_ids = final_df['peer_id'].astype(str).tolist()
        redundancies = final_df['message_redundancy'].tolist()

        # 创建Pyecharts条形图
        bar = (
            Bar()
            .add_xaxis(peer_ids)
            .add_yaxis("消息冗余数", redundancies, category_gap="50%")
            .set_global_opts(
                title_opts=opts.TitleOpts(
                    title=f"扇出 {fanout}: 第十分钟Peer消息冗余数",
                    subtitle=f"数据捕获时间: {actual_selected_ts.strftime('%Y-%m-%d %H:%M:%S.%f')}"
                ),
                xaxis_opts=opts.AxisOpts(name="Peer ID", axislabel_opts={"rotate": 45}),
                yaxis_opts=opts.AxisOpts(name="消息冗余数"),
                tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="shadow"),
                datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")]
            )
        )
        
        chart_path = os.path.join(output_dir, f"redundancy_fanout_{fanout}.html")
        bar.render(chart_path)
        print(f"图表已保存至: {chart_path}\n")

def plot_fanout_redundancy_relationship(fanouts, output_dir="redundancy_charts"):
    """
    探究扇出和消息冗余条数的关系，绘制折线图。
    计算每个扇出值下，第十分钟所有peers的平均消息冗余数。
    """
    os.makedirs(output_dir, exist_ok=True)
    print(f"图表将保存到目录: {os.path.abspath(output_dir)}\n")

    fanout_data = [] # 存储 (fanout, average_redundancy) 对

    for fanout in fanouts:
        filename = f"redundancy_data_fanout_{fanout}.csv"
        print(f"正在处理文件: {filename}...")

        result = get_10min_data_for_file(filename)
        if result is None:
            print(f"跳过文件 {filename}。\n")
            continue
        
        final_df, actual_selected_ts = result
        
        # 计算该扇出值下所有peers在第十分钟的平均消息冗余数
        average_redundancy = final_df['message_redundancy'].mean()
        fanout_data.append((fanout, average_redundancy))
        print(f"扇出 {fanout} 的平均消息冗余数: {average_redundancy:.2f} (数据捕获时间: {actual_selected_ts.strftime('%H:%M:%S.%f')})\n")

    if not fanout_data:
        print("没有足够的数据来生成扇出与消息冗余数的关系图。请检查文件是否存在且包含数据。")
        return

    # 按扇出值排序，确保折线图的X轴顺序正确
    fanout_data.sort(key=lambda x: x[0])
    fanouts_for_plot = [item[0] for item in fanout_data]
    avg_redundancies_for_plot = [item[1] for item in fanout_data]

    # 创建Pyecharts折线图
    line = (
        Line()
        .add_xaxis(fanouts_for_plot)
        .add_yaxis("平均消息冗余数", avg_redundancies_for_plot, is_smooth=True)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="扇出与平均消息冗余数的关系"),
            xaxis_opts=opts.AxisOpts(name="扇出值 (Fanout)", type_="value"),
            yaxis_opts=opts.AxisOpts(name="平均消息冗余数"),
            tooltip_opts=opts.TooltipOpts(trigger="axis"),
            datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")]
        )
    )
    
    chart_path = os.path.join(output_dir, "fanout_redundancy_relationship.html")
    line.render(chart_path)
    print(f"扇出与消息冗余数关系图已保存至: {chart_path}\n")

if __name__ == "__main__":
    fanout_values = [1, 3, 5, 7, 9] # 假设你的文件是 redundancy_data_fanout_X.csv
    
    # 你可以先运行第一个函数，生成每个扇出的peer冗余度图
    plot_redundancy_at_10min(fanout_values)
    
    # 然后运行第二个函数，生成扇出与平均冗余度的关系图
    plot_fanout_redundancy_relationship(fanout_values)

图表将保存到目录: d:\kk\Desktop\CS305-2025Spring-FinalProject_real\redundancy_charts

正在处理文件: redundancy_data_fanout_1.csv...
图表已保存至: redundancy_charts\redundancy_fanout_1.html

正在处理文件: redundancy_data_fanout_3.csv...
图表已保存至: redundancy_charts\redundancy_fanout_3.html

正在处理文件: redundancy_data_fanout_5.csv...
图表已保存至: redundancy_charts\redundancy_fanout_5.html

正在处理文件: redundancy_data_fanout_7.csv...
图表已保存至: redundancy_charts\redundancy_fanout_7.html

正在处理文件: redundancy_data_fanout_9.csv...
图表已保存至: redundancy_charts\redundancy_fanout_9.html

图表将保存到目录: d:\kk\Desktop\CS305-2025Spring-FinalProject_real\redundancy_charts

正在处理文件: redundancy_data_fanout_1.csv...
扇出 1 的平均消息冗余数: 25.09 (数据捕获时间: 23:57:24.148011)

正在处理文件: redundancy_data_fanout_3.csv...
扇出 3 的平均消息冗余数: 507.18 (数据捕获时间: 00:02:50.459015)

正在处理文件: redundancy_data_fanout_5.csv...
扇出 5 的平均消息冗余数: 868.18 (数据捕获时间: 09:18:35.163315)

正在处理文件: redundancy_data_fanout_7.csv...
扇出 7 的平均消息冗余数: 1191.82 (数据捕获时间: 12:57:09.249970)

正在处理文件: redundancy_data_fano